# Together.AI Client class

[Together.AI](https://www.together.ai/) is a cloud based platform serving many open-weight LLMs such as Google's Gemma, Meta's Llama 2/3, Qwen, Mistral.AI's Mistral, and Mixtral, NousResearch's Hermes models.

You can use AutoGen with [Together.AI's API directly](cloud-togetherai) as it is similar to the OpenAI API, however this client class uses their Python library which provides a more robust method to utilise their service.

## Features

The primary benefit of using the Together.AI API library, as this client class does, is messages can be tailored to accommodate the specific requirements of Together.AI's API while providing native support for function/tool calling, token usage, and accurate costs (as of June 2024).

## Getting started

First you need to install the `pyautogen` package to use AutoGen with the Together.AI API library.

``` bash
pip install pyautogen[together]
```

Together provides a large number of models to use, included some below. See the list of [models here](https://docs.together.ai/docs/inference-models).

See the sample `OAI_CONFIG_LIST` below showing how the Together.AI client class is used by specifying the `api_type` as `together`.

```python
[
    {
        "model": "gpt-35-turbo",
        "api_key": "your OpenAI Key goes here",
    },
    {
        "model": "gpt-4-vision-preview",
        "api_key": "your OpenAI Key goes here",
    },
    {
        "model": "dalle",
        "api_key": "your OpenAI Key goes here",
    },
    {
        "model": "google/gemma-7b-it",
        "api_key": "your Together.AI API Key goes here",
        "api_type": "together"
    },
    {
        "model": "codellama/CodeLlama-70b-Instruct-hf",
        "api_key": "your Together.AI API Key goes here",
        "api_type": "together"
    },
    {
        "model": "meta-llama/Llama-2-13b-chat-hf",
        "api_key": "your Together.AI API Key goes here",
        "api_type": "together"
    },
    {
        "model": "Qwen/Qwen2-72B-Instruct",
        "api_key": "your Together.AI API Key goes here",
        "api_type": "together"
    }
]
```

As an alternative to the `api_key` key and value in the config, you can set the environment variable `TOGETHER_API_KEY` to your Mistral AI key.

Linux/Mac:
``` bash
export TOGETHER_API_KEY="your Together.AI API key here"
```

Windows:
``` bash
set TOGETHER_API_KEY=your_together_ai_api_key_here
```

## API parameters

The following parameters can be added to your config for the Together.AI API. See [this link](https://docs.together.ai/reference/chat-completions) for further information on their purpose, default values, and ranges.

- max_tokens (integer)
- temperature (float)
- top_p (float)
- top_k (integer)
- repetition_penalty (float)
- frequency_penalty (float)
- presence_penalty (float)
- min_p (float)
- safety_model (string - [moderation models here](https://docs.together.ai/docs/inference-models#moderation-models))

Example:
```python
[
    {
        "model": "microsoft/phi-2",
        "api_key": "your Together.AI API Key goes here",
        "api_type": "together",
        "max_tokens": 1000,
        "stream": False,
        "temperature": 1,
        "top_p": 0.8,
        "top_k": 50,
        "repetition_penalty": 0.5,
        "presence_penalty": 1.5,
        "frequency_penalty": 1.5,
        "min_p": 0.2,
        "safety_model": "Meta-Llama/Llama-Guard-7b"
    }
]
```

## Two-Agent Coding Example

In this example, we run a two-agent chat with an AssistantAgent (primarily a coding agent) to generate code to count the number of prime numbers between 1 and 10,000 and then it will be executed.

We'll use Mistral's Mixtral-8x7B instruct model which is suitable for coding.

In [1]:
import os

config_list = [
    {
        # Let's choose the Mixtral 8x7B model
        "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
        # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        "stream": False,
    }
]

Importantly, we have tweaked the system message so that the model doesn't return the termination keyword, which we've changed to FINISH, with the code block.

In [2]:
from pathlib import Path

from autogen import AssistantAgent, UserProxyAgent
from autogen.coding import LocalCommandLineCodeExecutor

# Setting up the code executor
workdir = Path("coding")
workdir.mkdir(exist_ok=True)
code_executor = LocalCommandLineCodeExecutor(work_dir=workdir)

# Setting up the agents

# The UserProxyAgent will execute the code that the AssistantAgent provides
user_proxy_agent = UserProxyAgent(
    name="User",
    code_execution_config={"executor": code_executor},
    is_termination_msg=lambda msg: "FINISH" in msg.get("content"),
)

system_message = """You are a helpful AI assistant who writes code and the user executes it.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) for the user to execute.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
IMPORTANT: Wait for the user to execute your code and then you can reply with the word "FINISH". DO NOT OUTPUT "FINISH" after your code block."""

# The AssistantAgent, using Together.AI's Code Llama model, will take the coding request and return code
assistant_agent = AssistantAgent(
    name="Together Assistant",
    system_message=system_message,
    llm_config={"config_list": config_list},
)

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Start the chat, with the UserProxyAgent asking the AssistantAgent the message
chat_result = user_proxy_agent.initiate_chat(
    assistant_agent,
    message="Provide code to count the number of prime numbers from 1 to 10000.",
)

User (to Together Assistant):

Provide code to count the number of prime numbers from 1 to 10000.

--------------------------------------------------------------------------------
Together Assistant (to User):

 ```python
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

count = 0
for num in range(1, 10001):
    if is_prime(num):
        count += 1

print(count)
```

This code defines a helper function `is_prime(n)` to check if a number `n` is prime. It then iterates through numbers from 1 to 10000, checks if each number is prime using the helper function, and increments a counter if it is. Finally, it prints the total count of prime numbers found.

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
User (to Together Assist

## Tool Call Example

In this example, instead of writing code, we will have two agents playing chess against each other using tool calling to make moves.

We'll change models to the Qwen 2 72B model for this challenging task.

In [4]:
config_list = [
    {
        # Let's choose Meta's CodeLlama 34b instruct model which supports function calling through the Together.AI API
        "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        "api_type": "together",
    }
]


First install the `chess` package by running the following command:

In [5]:
! pip install chess

Defaulting to user installation because normal site-packages is not writeable


Write the function for making a move.

In [6]:
import random

import chess
import chess.svg
from IPython.display import display
from typing_extensions import Annotated

board = chess.Board()


def make_move() -> Annotated[str, "A move in UCI format"]:
    moves = list(board.legal_moves)
    move = random.choice(moves)
    board.push(move)
    # Display the board.
    display(chess.svg.board(board, size=400))
    return str(move)

Let's create the agents. We have three different agents:
- `player_white` is the agent that plays white.
- `player_black` is the agent that plays black.
- `board_proxy` is the agent that moves the pieces on the board.

In [7]:
from autogen import ConversableAgent, register_function

player_white = ConversableAgent(
    name="Player White",
    system_message="You are a chess player and you play as white. " "Always call make_move() to make a move",
    llm_config={"config_list": config_list, "cache_seed": None},
)

player_black = ConversableAgent(
    name="Player Black",
    system_message="You are a chess player and you play as black. " "Always call make_move() to make a move",
    llm_config={"config_list": config_list, "cache_seed": None},
)

board_proxy = ConversableAgent(
    name="Board Proxy",
    llm_config=False,
    # The board proxy will only respond to the make_move function.
    is_termination_msg=lambda msg: "tool_calls" not in msg,
)

Register tools for the agents. See the [tutorial chapter on tool use](/docs/tutorial/tool-use) 
for more information.

In [8]:
register_function(
    make_move,
    caller=player_white,
    executor=board_proxy,
    name="make_move",
    description="Make a move.",
)

register_function(
    make_move,
    caller=player_black,
    executor=board_proxy,
    name="make_move",
    description="Make a move.",
)

/home/autogen/autogen/autogen/agentchat/conversable_agent.py:2408: UserWarning: Function 'make_move' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


Register nested chats for the player agents.
Nested chats allows each player agent to chat with the board proxy agent
to make a move, before communicating with the other player agent.
See the [nested chats tutorial chapter](/docs/tutorial/conversation-patterns#nested-chats)
for more information.

In [9]:
player_white.register_nested_chats(
    trigger=player_black,
    chat_queue=[
        {
            "sender": board_proxy,
            "recipient": player_white,
        }
    ],
)

player_black.register_nested_chats(
    trigger=player_white,
    chat_queue=[
        {
            "sender": board_proxy,
            "recipient": player_black,
        }
    ],
)

Clear the board and start the chess game.

In [ ]:
# Clear the board.
board = chess.Board()

chat_result = player_white.initiate_chat(
    player_black,
    message="Let's play chess! Your move.",
    max_turns=4,
)